# Reading TUFLOW data 
In this script we are experimenting with reading in TUFLOW data to then load it into an xml file for FM.

In [ ]:
# loading modules:
import pathlib 
import sys, mmap, glob, os
import geopandas as gpd
import pyproj, fiona
import math

sys.path.append(r"C:\Users\phillio\Github\Open_source\floodmodeller-api")

In [ ]:
# loading .tgc file, will be useful for some data:
# print(pathlib.Path.cwd())
# tgcfile_path = pathlib.Path('C:/Users/phillio/OneDrive - Jacobs/Documents/TUFLOW_examples/Bootle/Bootle/TUFLOW/Model/BOOT_023.tgc')
# with open(tgcfile_path, "r") as tgcfile:
#     tgc_file = tgcfile.read()

# Reading the tgc file in a more robust way
Here we will be reading the tgc file line by line in a more robust way. Will be attempting to follow the structure from ief, self._read file by reading line by line and setting something = something else. Will be more of a challenfe to remove the comments

In [ ]:
tgcfile_path = pathlib.Path('C:/Users/phillio/OneDrive - Jacobs/Documents/TUFLOW_examples/Bootle/Bootle/TUFLOW/Model/BOOT_023.tgc')
with open(tgcfile_path, "r") as tgcfile:
    raw_data = [line.rstrip("\n") for line in tgcfile.readlines()]

raw_data

In [ ]:
#Now we need to clean the data
# start by removing any comment only lines
for line in raw_data:
    if line.lstrip().startswith("!"):
        raw_data.remove(line)
for line in raw_data:
    if line.lstrip().startswith("! "):
        raw_data.remove(line)

raw_data

In [ ]:
# now need to try and remove comments at the end of comments and empty space

#let us now replace the \t with empty spaces
raw_data = [item.replace('\t', '') for item in raw_data]

raw_data



In [ ]:
for line in range(len(raw_data)):
    # print(line)
    # print(raw_data[line])
    # print(raw_data[line].partition('!'))
    line_partition = raw_data[line].partition('!')
    raw_data[line] = line_partition[0] # only taking information from the left habd side if the '!'
    
raw_data

In [ ]:
# now we want to split the left hand side and the right hand side, I guess maybe we should remove any double spaces? We'll see how this is effected after this step
tgc_data = []
for line in raw_data:
    # print(line)
    if '==' in line:
        prop, value = [itm.strip() for itm in line.split('==', 1)]

        tgc_data.append((prop, value))

tgc_data

# Success I think??

We have now managed to get a list of tuples which have the values we want. These may need to be split in a more precise fashion or stored as different variable types but we should now be able to access ncols, nrows and step size

In [ ]:
# first trying to isolate the grid size

for line in range(len(tgc_data)):
    if tgc_data[line][0] == 'Cell Size':
        dx = float(tgc_data[line][1])
    
    if tgc_data[line][0] == 'Grid Size (X,Y)':
        line_partition = tgc_data[line][1].partition(',')
        n_X = float(line_partition[0])
        n_Y = float(line_partition[2])

(dx, n_X, n_Y)

In [ ]:
tgc_data[2][1]


# Playing with the loc line to access the initial points

In [ ]:
# loading in data for the orientation of domain line as a geopandas df
try:
    BOOT_L_015_file = pathlib.Path('C:/Users/phillio/OneDrive - Jacobs/Documents/TUFLOW_examples/Bootle/Bootle/TUFLOW/Model/gis/2d_loc_BOOT_L_015.shp')
    BOOT_L_015_file.open('a')
    df_BOOT_L_015 = gpd.read_file(BOOT_L_015_file)
except Exception as e:
    print(e)

df_BOOT_L_015

In [ ]:
# looking at the data
df_BOOT_L_015.info()

In [ ]:
df_BOOT_L_015.head()

In [ ]:
a = df_BOOT_L_015.geometry[0]

In [ ]:
x1, y1 = a.coords[0]
x2, y2 = a.coords[-1]

theta = math.atan2(y1-y2, x1- x2)  # check documentation

theta = math.degrees(theta)
print(theta)

In [ ]:
df_BOOT_L_015.plot()

In [ ]:
df_BOOT_L_015.geometry.boundary.bounds

## Have access to the orientation line data!

Question is now how can we compute the angle between this and the north south line, we should be able to take advatnage of some of the properties of the geopandas/shapely/fiona packages.

Otherwise, we could construct a vector and then do the cross product between them to find the angle.

## Finding the bounds of the boundary box
In this section we will be looking into the idea of finding the boundary box to find the lower left coordinates

In [ ]:
try:
    BOOT_L_023_file = pathlib.Path(r"C:\Users\phillio\OneDrive - Jacobs\Documents\TUFLOW_examples\Bootle\Bootle\TUFLOW\Model\gis\2d_bc_BOOT_R_023.shp")
    BOOT_L_023_file.open('a')
    df_BOOT_L_023 = gpd.read_file(BOOT_L_023_file)
except Exception as e:
    print(e)

df_BOOT_L_023

In [ ]:
df_BOOT_L_023.plot()

In [ ]:
try:
    BOOT_L_017_file = pathlib.Path(r"C:\Users\phillio\OneDrive - Jacobs\Documents\TUFLOW_examples\Bootle\Bootle\TUFLOW\Model\gis\2d_code_BOOT_R_017.shp")
    BOOT_L_017_file.open('a')
    df_BOOT_L_017 = gpd.read_file(BOOT_L_017_file)
except Exception as e:
    print(e)

df_BOOT_L_017



In [ ]:
df_BOOT_L_017.plot()

## Use df_BOOT_L_017 to construct boundary box
I believe this is correct, it certainly looks like a good starting point to draw a box around

In [ ]:
df_BOOT_L_017.geometry.boundary[0]

In [ ]:
df_BOOT_L_017["area"] = df_BOOT_L_017.area
df_BOOT_L_017.plot()
# df_BOOT_L_017.explore("area", legend=False)
df_BOOT_L_017.crs

In [ ]:
# test_df_BOOT_17 = df_BOOT_L_017.to_crs("EPSG:4326")
# test_df_BOOT_17.plot()
# test_df_BOOT_17.crs

In [ ]:
df_BOOT_L_017.geometry.bounds

In [ ]:
df_BOOT_L_017.geometry.bounds

In [ ]:

df17_envelope = df_BOOT_L_017.envelope
df17_envelope.geometry.plot()
df17_envelope.geometry.bounds

In [ ]:
xll = df17_envelope.geometry.bounds["minx"]
yll = df17_envelope.geometry.bounds["miny"]

print(xll)
print(yll)

# Some success!

We have now isolated the lower left coordinates!

# Slight problem!!
While we have successfully isolated the lower coordiantes, this is of the rotated dataframe, we need to rotate it back then perform this routine so the idea is here just not quite the correct execution.

code we need to use is df.gemetry.rotate(angle) see documentation: https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.rotate.html

Still need the angle, then should be able to repeat the above process to find the correct boundary box.

## maybe we can get away with this ...
It depends where we are rotating above, but I feel that we are probably roating around the bottom corner?

310279.406408	487446.868549

# Old method, not working!

# Reading the raw text file to access the grid spacing

So we need to be able to read the following information:

 Grid Definition
Grid Size (X,Y) == 2450, 1400																						! Grid dimensions (metres)
Cell Size == 2	

Specifically the grid size and cell size

First step is to try and read the numbers 2450, 1400

Probably easier to attempt to read the grid size 2 first!

## Reading grid size 2


In [ ]:
# reading grid size 2
type(tgc_file)
# grid_size_test = tgc_file.split('Cell Size == ')
# print(grid_size_test[0])

#idea, partition around Grid Size (X,Y) == and ! Grid dimensions (metres)
# then again around ! Grid dimensions (metres) and ! Cell size (metres)

str_partition1 = tgc_file.partition('! Cell size (metres)')
str_partition2 = str_partition1[0].partition('! Grid dimensions (metres)')
str_partition3 = str_partition2[0].partition('! Grid Definition')

# print(str_partition2[2])
# print(str_partition3[2])

str_partition4 = str_partition2[2].partition('Cell Size == ')
str_partition5 = str_partition3[2].partition('Grid Size (X,Y) == ')
str_partition6 = str_partition5[2].partition(', ')

# print(str_partition6[0])
# print(len(str_partition6[0]))

# print(str_partition6[2])
# len(str_partition6[2])

# print(str_partition5[0])
# print(str_partition5[1])
# print(str_partition5[2])

# test_1400 = float(str_partition6[2])
# print(len(str_partition5[0]))
# print(len(str_partition5[1]))
# print(len(str_partition5[2]))

# print(test_1400)
# print(str_partition7[0])
# print(len(str_partition7[0]))

Grid_size = float(str_partition4[2])
N_X = float(str_partition6[0])
N_Y = float(str_partition6[2])

print(Grid_size, N_X, N_Y)
print(type(Grid_size)) 
print(type(N_X))
print(type(N_Y))
# print(Grid_size)
# print(ncols_nrows)

# type(Grid_size)
# # type(ncols_nrows)
# len(str_partition5[2])
